# Preparing inputs and variables

## Import and input/output

In [ ]:
#imports
from osgeo import gdal
import os
import glob
import arcpy
from arcpy import env
from arcpy.sa import *

## Folder paths

In [ ]:
# Folder path
folder_path = os.path.dirname(os.path.abspath("__file__"))

#import dir
iDirname = r"{0}\import".format(folder_path)
#export dir
eDirname = r"{0}\export".format(folder_path)

print(folder_path,iDirname,eDirname,sep='\n')

## Variables

In [ ]:
raster_year = "2021"
raster_month = "09"
raster_day = "10"
raster_date = "compbands_20210910"
compbands_tif = "compbands_20210910.tif"

#--------#
#raster_year = "2021"
#raster_month = "09"
#raster_day = "30"
#raster_date = "compbands_20210930"
#compbands_tif = "compbands_20210930.tif"

#-------#
#raster band spatial resolution
R10m_res = "R10m"
R20m_res = "R20m"

In [ ]:
# Allow reprojection
# change to variable to EPSG code wanted export reprojection tif.
EPSG_projection = "none"

In [ ]:
# Allow clipping
# set TRUE or FALSE for clipping
clip_allow = "TRUE"
clip_json = "La_palma_bounds.geojson"
clip_shp = "La_palma_bounds.shp"

In [ ]:
# Allow normalization
# set TRUE or FALSE for normalization
sr_allow = "FALSE"
sr_input =eDirname + "\\" +compbands_tif
sr_output = eDirname + "\\" +raster_date+"_sr.tif"

In [ ]:
# Allow resampling
# set TRUE or FALSE for resampling
resampling_allow = "FALSE"
resampling_input = compbands_tif
resampling_output = "resamp_"+raster_date
resampling_pixel_size = "10"
resampling_calc = "NEAREST"

# Functions

In [ ]:
# Reprojection function
def raster_reprojection(raster_input, raster_output, EPSG_projection):
    input_raster = arcpy.Raster(raster_input)

    # reproject the input raster to WGS 1984 UTM Zone 11N 
    reprojected_raster = arcpy.ia.Reproject(input_raster, {"wkid" : EPSG_projection}) # 32628 WGS 84 / UTM zone 28N

    # verify the new coordinate system
    prj = print(arcpy.Describe(reprojected_raster).spatialReference.name)

    # save the output
    reprojected_raster.save(raster_output)
    print("reprojection finished")

def raster_clip(inRaster,inMaskData,extraction_area, out_feature_class):
    # Execute ExtractByMask
    outExtractByMask = arcpy.sa.ExtractByMask(inRaster, inMaskData, extraction_area)
    # Save the output 
    outExtractByMask.save(out_feature_class)
    print("clip finished")
    
# NORMALIZE to surface reflectance values
# Multiplying the DN by 10000 on sentinel-2 images, convert the values to Surface reflectance.
# With surface reflectance, further remote sensing analysis can be done: land-use classification, spectrun signatures, etc.
def surface_reflectance(inRaster, outRaster):
    outTimes = Raster(inRaster) * 10000
    outTimes.save(outRaster)
    print("Normalization is TRUE")

# Sentinel-2

In [ ]:
# images bands folder path
path_raster_bands_10m = glob.glob("{0}\*{1}{2}{3}*.SAFE\GRANULE\L2A_*\IMG_DATA\{4}\*.jp2*".format(iDirname, raster_year,
                                                                               raster_month, raster_day, R10m_res))

path_raster_bands_20m = glob.glob("{0}\*{1}{2}{3}*.SAFE\GRANULE\L2A_*\IMG_DATA\{4}\*.jp2*".format(iDirname, raster_year,
                                                                               raster_month, raster_day, R20m_res))

### Select bands

In [ ]:
#Sentinel bands folder paths
b2_10m = next((s for s in path_raster_bands_10m if "_B02_" in s), None)
b3_10m = next((s for s in path_raster_bands_10m if "_B03_" in s), None)
b4_10m = next((s for s in path_raster_bands_10m if "_B04_" in s), None)
b8_10m = next((s for s in path_raster_bands_10m if "_B08_" in s), None)
b11_20m = next((s for s in path_raster_bands_20m if "_B11_" in s), None)

print(b2_10m)

# Set ArcGIS enviroment

In [ ]:
# SET ARCGIS ENVIRONMENT
arcpy.env.workspace = eDirname
# Output fields are unqualified, so the field name will not contain the origin table
arcpy.env.qualifiedFieldNames = False
arcpy.env.scratchWorkspace = eDirname
#Allow for overwriting
arcpy.env.overwriteOutput = True

## Resampling function
Note that the following function "compose multi band raster", automatically resample the pixel size according to the first image input.
For this reason, this function is not needed

In [ ]:
# Resample function
if resampling_allow == "TRUE":
    arcpy.Resample_management(resampling_input, resampling_output, resampling_pixel_size, resampling_calc)

## Compose multi band raster

In [ ]:
# 1) Compose multi types of single band raster datasets to a TIFF format raster dataset
arcpy.CompositeBands_management(b2_10m+";"+b3_10m+";"+b4_10m+";"+b8_10m+";"+b11_20m, compbands_tif)

## Check metadata

In [ ]:
# open raster tif
ds = gdal.Open(eDirname + "\\" + compbands_tif)
array = ds.GetRasterBand(1).ReadAsArray()

In [ ]:
# 2) check metadata
print(ds.GetGeoTransform())
print(ds.GetProjection())

## Reprojection

In [ ]:
if EPSG_projection != "none":
    input_raster = eDirname + "\\" + compbands_tif
    raster_output = eDirname + "\\" + "reproj_"+compbands_tif
    EPSG_projection = EPSG_projection

    raster_reprojection(raster_input, raster_output, EPSG_projection)

## Clip image

In [ ]:
if clip_allow == "TRUE":
    # Convert geojson to shp
    arcpy.conversion.JSONToFeatures(iDirname +"\\"+clip_json, os.path.join(eDirname, clip_shp))
    print("json converted to shp")
    
    #variables
    inRaster = eDirname + "\\" + compbands_tif
    inMaskData = eDirname + "\\" +clip_shp
    extraction_area = "INSIDE"
    out_feature_class = eDirname + "\\" +"clip_"+compbands_tif
    #Call function
    raster_clip(inRaster, inMaskData, extraction_area, out_feature_class)



## Normalizing of DN to surface reflectance

In [ ]:
if sr_allow == "TRUE":
    surface_reflectance(sr_input, sr_output)

In [ ]:
print("FINISHED ANALYSIS")